# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
try:
    df_1 = pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/geo_data_0.csv')
except:
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')

In [3]:
try:
    df_2 = pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/geo_data_1.csv')
except:
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')

In [4]:
try:
    df_3 = pd.read_csv('C:/Users/Пользователь/OneDrive/Документы/Python/geo_data_2.csv')
except:
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')

Сначала посмотрим наобщую информацию и проверим на дубликаты.

In [5]:
df_1.info()
df_2.info()
df_3.info()# Посмотрим общую информацию

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [6]:
df_1.head()# взглянем на первые 5 строчек по каждому региону

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [7]:
df_2.head()# взглянем на первые 5 строчек по каждому региону

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [8]:
df_3.head()# взглянем на первые 5 строчек по каждому региону

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [9]:
df_1.isna().sum()# посмотрим количество пропущенных значений для каждого столбца

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [10]:
df_2.isna().sum()# посмотрим количество пропущенных значений для каждого столбца

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [11]:
df_3.isna().sum()# посмотрим количество пропущенных значений для каждого столбца

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
df_1.duplicated().sum()# проверим наши исходные данные на явные дубликаты

0

In [13]:
df_2.duplicated().sum()# проверим наши исходные данные на явные дубликаты

0

In [14]:
df_3.duplicated().sum()# проверим наши исходные данные на явные дубликаты

0

In [15]:
df_1['id'].duplicated().sum()# проверим наши исходные данные на неявные дубликаты

10

In [16]:
df_1 = df_1.drop_duplicates (subset=['id'])# удалим дубликаты

In [17]:
df_2['id'].duplicated().sum()# проверим наши исходные данные на неявные дубликаты

4

In [18]:
df_2 = df_2.drop_duplicates (subset=['id'])# удалим дубликаты

In [19]:
df_3['id'].duplicated().sum()# проверим наши исходные данные на неявные дубликаты

4

In [20]:
df_3 = df_3.drop_duplicates (subset=['id'])# удалим дубликаты

Найденных дубликатов совсем немного, но для чистоты эксперемента их лучше удалить.

In [21]:
df_1 = df_1.drop(['id'], axis=1)# удалим ненужные для модели столбцы
df_2 = df_2.drop(['id'], axis=1)# удалим ненужные для модели столбцы
df_3 = df_3.drop(['id'], axis=1)# удалим ненужные для модели столбцы

Данные проверены и готовы к дальнейшим действиям.

## Обучение и проверка модели

Поделим данные на обучающую и валидационную выборки.

In [22]:
df_1_train, df_1_valid = train_test_split(df_1, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки
df_2_train, df_2_valid = train_test_split(df_2, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки
df_3_train, df_3_valid = train_test_split(df_3, test_size=0.25, random_state=12345) # отделим 25% данных для валидационной выборки

In [23]:
features_1 = df_1_train.drop(['product'], axis=1)
target_1 = df_1_train['product']
features_valid_1 = df_1_valid.drop(['product'], axis=1)
target_valid_1 = df_1_valid['product']

features_2 = df_2_train.drop(['product'], axis=1)
target_2 = df_2_train['product']
features_valid_2 = df_2_valid.drop(['product'], axis=1)
target_valid_2 = df_2_valid['product']

features_3 = df_3_train.drop(['product'], axis=1)
target_3 = df_3_train['product']
features_valid_3 = df_3_valid.drop(['product'], axis=1)
target_valid_3 = df_3_valid['product']

Обучим модели и посмотрим на средний запас сырья и RMSE.

In [24]:
model_1 = LinearRegression()# применим метод
model_1.fit(features_1, target_1)# обучим модель
predicted_1 = model_1.predict(features_valid_1)
np_array_1 = np.array(predicted_1)
new_predicted_1 = pd.Series(np_array_1, index = target_valid_1.index)
rmse_1 = mean_squared_error(target_valid_1, predicted_1) ** 0.5
print('RMSE :',rmse_1)
print('Средний запас сырья :', predicted_1.mean())

RMSE : 37.853527328872964
Средний запас сырья : 92.78915638280621


In [25]:
model_2 = LinearRegression()# применим метод
model_2.fit(features_2, target_2)# обучим модель
predicted_2 = model_2.predict(features_valid_2)
np_array_2 = np.array(predicted_2)
new_predicted_2 = pd.Series(np_array_2, index = target_valid_2.index)
rmse_2 = mean_squared_error(target_valid_2, predicted_2) ** 0.5
print('RMSE :',rmse_2)
print('Средний запас сырья :', predicted_2.mean())

RMSE : 0.892059264771703
Средний запас сырья : 69.17831957030432


In [26]:
model_3 = LinearRegression()# применим метод
model_3.fit(features_3, target_3)# обучим модель
predicted_3 = model_3.predict(features_valid_3)
np_array_3 = np.array(predicted_3)
new_predicted_3 = pd.Series(np_array_3, index = target_valid_3.index)
rmse_3 = mean_squared_error(target_valid_3, predicted_3) ** 0.5
print('RMSE :',rmse_3)
print('Средний запас сырья :', predicted_3.mean())

RMSE : 40.07585073246016
Средний запас сырья : 94.86572480562035


Самый низкий запас сырья во втором регионе, но метрика RMSE выдает показатель значительно лучше первого и третьего регионов.

## Подготовка к расчёту прибыли

- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

In [27]:
BUDGET = 10000000000
BARELL_PRICE = 450000
BOREHOLE = 500
TOP_BOREHOLE = 200

In [28]:
WITHOUT_lOSS = BUDGET/BARELL_PRICE/TOP_BOREHOLE
WITHOUT_lOSS

111.11111111111111

Что бы добится безубыточной разработки минимальное среднее значение запасов сырья в каждой из 200 лучших скважин региона должна ровняться 111.11111111111111.
Напомню общее среднее со всех скважин в регионе : 92.78915638280621, 69.1783195703043, 94.86572480562035.

## Расчёт прибыли и рисков 

Вся предварительная работа проделана. Пора уже выяснить какой же регион является предпочтительным для разработки.

In [29]:
def oil(target, predictions, price, count):
    pred_sorted = predictions.sort_values(ascending=False)
    pred_count = target[pred_sorted.index][:count]
    return price * pred_count.sum() - BUDGET 

In [30]:
state = np.random.RandomState(12345)

In [31]:
values = []
for i in range(1000):
    target_i = target_valid_1.sample(replace = True, random_state = state, n = BOREHOLE)
    predicted_i = new_predicted_1[target_i.index]
    values.append(oil(target_i,predicted_i,BARELL_PRICE, TOP_BOREHOLE))

values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
    
print("Средняя выручка", values.mean())
print("Риск убыточности", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: от",lower,"до",upper)

Средняя выручка 409428038.62143606
Риск убыточности 7.1000000000000005
95%-й Доверительный интервал: от -131536028.70166382 до 944395582.7546725


In [32]:
values = []
for i in range(1000):
    target_i = target_valid_2.sample(replace = True, random_state = state, n = BOREHOLE)
    predicted_i = new_predicted_2[target_i.index]
    values.append(oil(target_i,predicted_i,BARELL_PRICE, TOP_BOREHOLE))

values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
    
print("Средняя выручка", values.mean())
print("Риск убыточности", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: от",lower,"до",upper)

Средняя выручка 536400199.43510306
Риск убыточности 0.30000000000000004
95%-й Доверительный интервал: от 112954247.12370124 до 998504156.6468805


In [33]:
values = []
for i in range(1000):
    target_i = target_valid_3.sample(replace = True, random_state = state, n = BOREHOLE)
    predicted_i = new_predicted_3[target_i.index]
    values.append(oil(target_i,predicted_i,BARELL_PRICE, TOP_BOREHOLE))

values = pd.Series(values)

lower = values.quantile(0.025)
upper = values.quantile(0.975)
    
print("Средняя выручка", values.mean())
print("Риск убыточности", st.percentileofscore(values, 0))
print("95%-й Доверительный интервал: от",lower,"до",upper)

Средняя выручка 339478034.1977997
Риск убыточности 11.8
95%-й Доверительный интервал: от -224089221.7440758 до 847067587.6863929


### Вывод

Итак мы провели исследование трех регинов для выявления самого потенциально прибыльного из них. И проверить его на риск убыточности перед вложением денег в разработку.
Получили следующие результаты:

Регион 1:
- Средний запас сырья : 92.78915638280621
- Средняя выручка 409428038.6214359
- Риск убыточности 7.1
- 95%-й Доверительный интервал: от -131536028.70166382 до 944395582.7546725

Регион 2:
- Средний запас сырья : 69.1783195703043
- Средняя выручка 536400199.43510294
- Риск убыточности 0.3
- 95%-й Доверительный интервал: от 112954247.12370124 до 998504156.6468805

Регион 3:
- Средний запас сырья : 94.86572480562035
- Средняя выручка 339478034.1977996
- Риск убыточности 11.8
- 95%-й Доверительный интервал: от -224089221.7440758 до 847067587.6863929


По получившимся данным видно, что несмотря на самый небольшой запас сырья - наиболее подходящим и единственным отвечающим предъявленным требованиям является регион 2!
Именно его и рекомендую к вложению средств и последующей разработке с минимальными рисками!